In [33]:
import numpy as np
import numpy.linalg as lng
import math
import time

def lduDecomp(A):

    '''
    return three arrays by decomposing matrix A

        Parameters
    ----------
    A   : ndarray
          the matrix to be decomposed
   
    Ruturn
    ----------
    L: ndarray
       Lower tringular elements of matrix of A, with zero in the diagonalse
    D: ndarray
       Diagonal elements of matrix of A
    U: ndarray
       Upper tringular elements of matrix A, with zero in the diagonals
       
    Description
    ----------
    The function will decompse matrix A in to three matrixs, upper and lower 
    tringular matrix, and a diagonal matrix
    '''
    n = A.shape[0] 
    L = np.zeros((n, n))
    D = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(0, n):
        L[i+1:, i] = A[i+1:, i]
        D[i, i]    = A[i, i]
        U[i, i+1:] = A[i, i+1:]
    return L,D,U

def spictralAnalysis(T):
    
    spictralRadius = np.amax(abs(np.linalg.eigvals(T)))
    
    if  spictralRadius >= 1:
        print('Spictral Radius =', spictralRadius)
        raise Exception("Iterative method won't converge, Spectral Radius >= 1")
    return spictralRadius


In [40]:
# A = np.array([[4, 1, 1, 23], [2, -9, 23, 34], [12, -8, -6, 62], [12, 34, 43, 5]])
b = np.array([12, 2, 4])

A = np.array([[40, 1, 1], [2, -9, 1], [10, -8, -6]])

# A  = np.zeros((10000, 10000))
L,D,U = lduDecomp(A)


In [41]:
for i in range(A.shape[0]):
    for j in range(A.shape[0]):
        if i > j : L[i, j] = A[i, j]
        if i == j: D[i, j] = A[i, j]
        if i < j : U[i, j] = A[i, j]

In [18]:
def jacobi(A, b, RTOL, ATOL):
    
    '''
    return an approximation to the solution of the linear system 
    by itreativly solving an intreation function

        Parameters
    ----------
    A    : NxN ndarray
           the matrix representing the coffients of the linear system
    b    : ndarray
           r.h.s vector
    RLOT : float
           Relative Tolorance
    ATOL : float
           Absolute Tolorance
   
    Ruturn
    ----------
    out: ndarray
         The approximated solution of Ax = b
       
    Description
    ----------
    The Jacobi method is a fixed point iteration solver where it general 
    form xk+1 = Txk + c where:
        T = D'(-L-U)
        c = D'S
    '''
    
    L,D,U = lduDecomp(A)
    S     = -L-U
    T     = np.dot(lng.inv(D), S)
    #Sanity check-------------------------------------------------------------
    spictralAnalysis(T)
    #-------------------------------------------------------------------------
    c = np.dot(lng.inv(D), b)

    x         = np.ones(A.shape[0])
    xNext     = np.zeros(A.shape[0])
    error     = lng.norm(x - xNext)
    iteration = 0
    
    
    while error > RTOL*lng.norm(x) + ATOL:
        xNext      = np.dot(T, x) + c
        error      = lng.norm(x - xNext)
        x          = xNext
        iteration += 1
    print('Iterations:', iteration)
    return xNext

In [20]:
def gaussSiedel(A, b, RTOL, ATOL):
    
    '''
    return an approximation to the solution of the linear system 
    by itreativly solving an intreation function

        Parameters
    ----------
    A    : NxN ndarray
           the matrix representing the coffients of the linear system
    b    : ndarray
           r.h.s vector
    RLOT : float
           Relative Tolorance
    ATOL : float
           Absolute Tolorance
   
    Ruturn
    ----------
    out: ndarray
         The approximated solution of Ax = b
       
    Description
    ----------
    The Gauss Siedel method is a fixed point iteration solver where it consider
    the next iteration in computing the solution it general form xk+1 = Txk + c 
    where:
        T = (D+L)'U
        c = (D+L)'b
    '''
    
    L,D,U = lduDecomp(A)
    T     = np.dot(-lng.inv(D + L), U)
    #Sanity check-------------------------------------------------------------
    spictralAnalysis(T)
    #-------------------------------------------------------------------------
    c = np.dot(lng.inv(D + L), b)
    
    x         = np.ones(A.shape[0])
    xNext     = np.zeros(A.shape[0])
    error     = lng.norm(x - xNext)
    iteration = 0
    
    while error > RTOL*lng.norm(x) + ATOL:
        xNext      = np.dot(T, x) + c
        error      = lng.norm(x - xNext)
        x          = xNext
        iteration += 1
    print('Iterations:', iteration)
    return xNext

In [24]:
def SOR(A, b, w, RTOL, ATOL):
    
    '''
    return an approximation to the solution of the linear system 
    by itreativly solving an intreation function

        Parameters
    ----------
    A    : NxN ndarray
           the matrix representing the coffients of the linear system
    b    : ndarray
           r.h.s vector
    RLOT : float
           Relative Tolorance
    ATOL : float
           Absolute Tolorance
   
    Ruturn
    ----------
    out: ndarray
         The approximated solution of Ax = b
       
    Description
    ----------
    The Succsive over relaxation method is a fixed point iteration solver 
    where it consider wieghted Gauss Siedel in computing the solution it 
    general form xk+1 = Txk + c where:
        T = (D+wL)'(-wU+(1-w)D)
        c = (D+wL)'(-wU+(1-w)D)
    '''
    
    L,D,U = lduDecomp(A)
    T     = np.dot(lng.inv(D + w*L),(-w*U + (1-w)*D))
    #Sanity check-------------------------------------------------------------
    spictralAnalysis(T)
    if w >= 2:
        raise Exception("SOR method won't converge for this weight w")
    #-------------------------------------------------------------------------
    c = np.dot(lng.inv(D + w*L), w*b) 
    
    x         = np.ones(A.shape[0])
    xNext     = np.zeros(A.shape[0])
    error     = lng.norm(x - xNext)
    iteration = 0
    
    while error > RTOL*lng.norm(x) + ATOL:
        xNext      = np.dot(T, x) + c
        error      = lng.norm(x - xNext)
        x          = xNext
        iteration += 1
    print('Iterations:', iteration)
    return xNext

In [42]:
x1 = jacobi(A, b, 1e-7, 1e-7)
print(x1)

x2 = gaussSiedel(A, b, 1e-7, 1e-7)
print(x2)

# w = 2/(1 + math.sqrt(1 - 0.4422353506166324**2))
x3 = SOR(A, b, 0.91201, 1e-7, 1e-7)
print(x3)

Iterations: 22
[ 0.30279503 -0.15062112  0.03881985]
Iterations: 10
[ 0.30279503 -0.15062113  0.03881989]
Iterations: 8
[ 0.30279503 -0.15062113  0.03881989]
